# Preparação e Estruturação dos Dados Brutos (Camada RAW)

Este notebook é responsável por ler os dados coletados na camada Landing (LND), aplicando esquemas estruturados para transformar os arquivos JSON e CSV em DataFrames do PySpark. Os dados são então organizados, renomeados e armazenados no formato Delta Lake na camada RAW, com particionamento para otimização do armazenamento e consultas futuras.

## Configuração e Importações do pyspark e delta lake

In [1]:
# Para iniciar a seção spark
from typing import Optional, Union, List
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql.functions import col
from delta import configure_spark_with_delta_pip
from delta.tables import DeltaTable

In [2]:
builder: SparkSession.Builder = SparkSession.builder \
    .appName("App Preparação RAW") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark: SparkSession = configure_spark_with_delta_pip(builder).getOrCreate()

spark

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/07/20 12:01:09 WARN Utils: Your hostname, BRALSOFT42, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/07/20 12:01:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/wilcb/projeto_data_warehouse/venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/wilcb/.ivy2.5.2/cache
The jars for the packages stored in: /home/wilcb/.ivy2.5.2/jars
io.delta#delta-spark_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c8cfabe0-e1c6-4ebc-a494-e513447bb6a1;1.0
	confs: [default]
	found io.delta#delta-spark_2.13;4.0.0 in central
	found io.delta#delta-storage;4.0.0 in central
	found org.antlr#antlr4-runtime;4.13.1 in central
:: resolution report :: resolve 273ms :: artifacts dl 11ms
	:: mod

## Funções para evitar repetição

In [6]:
# Apenas para arquivos com formato CSV.
def ler_csv(
        path: str,
        encoding: str,
        schema: Optional[StructType] = None,
        header: bool = False,
        sep: str = ";",
        quote: str = '"'
) -> DataFrame:
    """
    Lê um arquivo CSV usando PySpark com esquema definido e opções customizáveis.

    Esta função lê um arquivo CSV a partir do caminho especificado, aplicando o esquema
    fornecido para garantir tipos corretos nas colunas. Permite configurar o encoding,
    se o arquivo possui cabeçalho, o separador de campo e o caractere de aspas.

    Args:
        path (str): Caminho do arquivo CSV a ser lido.
        schema (StructType): Esquema (schema) PySpark para aplicar ao DataFrame.
        encoding (str): Codificação do arquivo CSV, ex: "UTF-8", "ISO-8859-1", etc.
        header (bool, optional): Indica se o arquivo possui linha de cabeçalho com nomes das colunas.
            Padrão é False.
        sep (str, optional): Caractere separador dos campos no CSV. Padrão é ponto e vírgula ";".
        quote (str, optional): Caractere usado para delimitar campos com texto. Padrão é aspas duplas '"'.

    Returns:
        DataFrame: DataFrame do Spark contendo os dados lidos do CSV conforme o esquema informado.

    """
    try:
        reader = spark.read.format("csv") \
            .option("header", header) \
            .option("sep", sep) \
            .option("quote", quote) \
            .option("encoding", encoding)

        if schema:
            reader = reader.schema(schema)
        else:
            reader = reader.option("inferSchema", True)

        df = reader.load(path)
        return df

    except Exception as e:
        print(f"Erro na leitura do CSV: {e}")

In [7]:
def gravar_delta(
    df: DataFrame,
    path: str,
    mode: str = "overwrite",
    partitionBy: Optional[Union[str, List[str]]] = None
) -> DataFrame:
    """
    Grava um DataFrame no formato Delta Lake em um caminho especificado.

    Esta função salva o DataFrame fornecido no formato Delta, com modo de gravação configurável
    e possibilidade de particionamento por uma ou mais colunas. Caso ocorra algum erro durante
    a gravação, a função captura a exceção e exibe uma mensagem de erro.

    Args:
        df (DataFrame): DataFrame do Spark a ser salvo.
        path (str): Caminho do diretório onde o Delta será gravado.
        mode (str, optional): Modo de gravação, como 'overwrite' (sobrescrever), 'append' (acrescentar),
            entre outros suportados pelo Spark. Padrão é 'overwrite'.
        partitionBy (str ou List[str], optional): Coluna ou lista de colunas para particionar os dados.
            Se None, não particiona. Padrão é None.

    Returns:
        DataFrame: O mesmo DataFrame passado como argumento, para permitir encadeamento de comandos.

    """
    try:
        writer = df.write.format("delta").mode(mode)

        if partitionBy:
            writer = writer.partitionBy(partitionBy)

        writer.save(path)
        print("Gravação efetuada!")
        return df

    except Exception as e:
        print(f"[ERRO] Falha ao gravar Delta no caminho '{path}': {e}")

## RAW - Dados IBGE

In [ ]:
# Caminho do arquivo
path_json_ibge_municipios: str = "../LND/municipios/municipios_2025-07-16.json"

In [92]:
# Definindo schema para o JSON aninhado dos municípios IBGE
schema_ibge_municipios: StructType = StructType([
    StructField("id", StringType(), True),
    StructField("nome", StringType(), True),
    StructField("microrregiao", StructType([
        StructField("id", StringType(), True),
        StructField("nome", StringType(), True),
        StructField("mesorregiao", StructType([
            StructField("id", StringType(), True),
            StructField("nome", StringType(), True),
            StructField("UF", StructType([
                StructField("id", StringType(), True),
                StructField("sigla", StringType(), True),
                StructField("nome", StringType(), True),
                StructField("regiao", StructType([
                    StructField("id", StringType(), True),
                    StructField("sigla", StringType(), True),
                    StructField("nome", StringType(), True),
                ]))
            ]))
        ]))
    ])),
    StructField("regiao-imediata", StructType([
        StructField("id", StringType(), True),
        StructField("nome", StringType(), True),
        StructField("regiao-intermediaria", StructType([
            StructField("id", StringType(), True),
            StructField("nome", StringType(), True),
            StructField("UF", StructType([
                StructField("id", StringType(), True),
                StructField("sigla", StringType(), True),
                StructField("nome", StringType(), True),
                StructField("regiao", StructType([
                    StructField("id", StringType(), True),
                    StructField("sigla", StringType(), True),
                    StructField("nome", StringType(), True),
                ]))
            ]))
        ]))
    ]))
])

In [ ]:
try:
    # Converter para DataFrame Spark
    df_ibge_municipios: DataFrame = spark.read \
        .option("multiline", True) \
        .schema(schema_ibge_municipios).json(path_json_ibge_municipios)
except Exception as e:
    print(f"Erro: {e}")

In [95]:
# Dados após seleção
df_ibge_municipios.show(5, truncate=False)

+-------+---------------------+----------------------------------------------------------------------------------------+---------------------------------------------------------------------------+
|id     |nome                 |microrregiao                                                                            |regiao-imediata                                                            |
+-------+---------------------+----------------------------------------------------------------------------------------+---------------------------------------------------------------------------+
|1100015|Alta Floresta D'Oeste|{11006, Cacoal, {1102, Leste Rondoniense, {11, RO, Rondônia, {1, N, Norte}}}}           |{110005, Cacoal, {1102, Ji-Paraná, {11, RO, Rondônia, {1, N, Norte}}}}     |
|1100023|Ariquemes            |{11003, Ariquemes, {1102, Leste Rondoniense, {11, RO, Rondônia, {1, N, Norte}}}}        |{110002, Ariquemes, {1101, Porto Velho, {11, RO, Rondônia, {1, N, Norte}}}}|
|1100031|Cabixi

In [ ]:
# Escrever dados com Delta Lake no formato parquet.

# gravar_delta(df_ibge_municipios, "../RAW/municipios")

Gravação efetuada!


DataFrame[id: string, nome: string, microrregiao: struct<id:string,nome:string,mesorregiao:struct<id:string,nome:string,UF:struct<id:string,sigla:string,nome:string,regiao:struct<id:string,sigla:string,nome:string>>>>, regiao-imediata: struct<id:string,nome:string,regiao-intermediaria:struct<id:string,nome:string,UF:struct<id:string,sigla:string,nome:string,regiao:struct<id:string,sigla:string,nome:string>>>>]

In [109]:
# Tentativa de carregar a tabela Delta salva na camada RAW para validar a gravação.
# Caso ocorra algum erro (por exemplo, caminho inválido ou problema no formato Delta), o erro será capturado e exibido.
try:
    deltaTable: DeltaTable = DeltaTable.forPath(spark, "../RAW/municipios")
    df_municipios: DataFrame = deltaTable.toDF()
    df_municipios.show(truncate=False)
except Exception as e:
    print(f"Erro: {e}")

+-------+------------------------+----------------------------------------------------------------------------------------+-----------------------------------------------------------------------------+
|id     |nome                    |microrregiao                                                                            |regiao-imediata                                                              |
+-------+------------------------+----------------------------------------------------------------------------------------+-----------------------------------------------------------------------------+
|1100015|Alta Floresta D'Oeste   |{11006, Cacoal, {1102, Leste Rondoniense, {11, RO, Rondônia, {1, N, Norte}}}}           |{110005, Cacoal, {1102, Ji-Paraná, {11, RO, Rondônia, {1, N, Norte}}}}       |
|1100023|Ariquemes               |{11003, Ariquemes, {1102, Leste Rondoniense, {11, RO, Rondônia, {1, N, Norte}}}}        |{110002, Ariquemes, {1101, Porto Velho, {11, RO, Rondônia, {1, N, Nor

## RAW - Dados CNAE

In [99]:
# Caminho para o CSV na camada LND
path_csv_cnae: str = "../LND/CSV/cnaes.csv"

In [100]:
# Schema esperado para o arquivo CNAE
schema_cnae: StructType = StructType([
    StructField("codigo_cnae", StringType(), True),
    StructField("descricao_cnae", StringType(), True)
])

In [105]:
# Leitura do arquivo CSV com o schema definido
df_cnae = ler_csv(path_csv_cnae, "ISO-8859-1", schema_cnae)

In [106]:
# Visualizar os dados carregados
df_cnae.show(truncate=False)

+-----------+-----------------------------------------------------------------------------------+
|codigo_cnae|descricao_cnae                                                                     |
+-----------+-----------------------------------------------------------------------------------+
|0111301    |Cultivo de arroz                                                                   |
|0111302    |Cultivo de milho                                                                   |
|0111303    |Cultivo de trigo                                                                   |
|0111399    |Cultivo de outros cereais não especificados anteriormente                          |
|0112101    |Cultivo de algodão herbáceo                                                        |
|0112102    |Cultivo de juta                                                                    |
|0112199    |Cultivo de outras fibras de lavoura temporária não especificadas anteriormente     |
|0113000    |Cultivo

In [ ]:
# Escrita no formato Delta na camada RAW

# gravar_delta(df_cnae, "../RAW/cnae")

Gravação efetuada!


DataFrame[codigo_cnae: string, descricao_cnae: string]

In [110]:
# Leitura da Delta Table salva para validação
try:
    deltaTable: DeltaTable = DeltaTable.forPath(spark, "../RAW/cnae")
    df_cnae: DataFrame = deltaTable.toDF()
    df_cnae.show(truncate=False)
except Exception as e:
    print(f"Erro: {e}")

+-----------+-----------------------------------------------------------------------------------+
|codigo_cnae|descricao_cnae                                                                     |
+-----------+-----------------------------------------------------------------------------------+
|0111301    |Cultivo de arroz                                                                   |
|0111302    |Cultivo de milho                                                                   |
|0111303    |Cultivo de trigo                                                                   |
|0111399    |Cultivo de outros cereais não especificados anteriormente                          |
|0112101    |Cultivo de algodão herbáceo                                                        |
|0112102    |Cultivo de juta                                                                    |
|0112199    |Cultivo de outras fibras de lavoura temporária não especificadas anteriormente     |
|0113000    |Cultivo

## RAW - Natureza Jurídica

In [ ]:
# Caminho para o arquivo CSV da natureza jurídica
path_csv_natureza_juridica: str = "../LND/CSV/natureza_juridica.csv"

In [113]:
# Definição do schema para o CSV de natureza jurídica
schema_natureza_juridica: StructType = StructType([
    StructField("codigo_natureza_juridica", StringType(), True),
    StructField("natureza_juridica", StringType(), True),
])

In [ ]:
# Leitura do arquivo CSV com o schema definido
df_natureza_juridica: DataFrame = ler_csv(path_csv_natureza_juridica, "iso-8859-1", schema_natureza_juridica)

In [117]:
# Visualização inicial dos dados
df_natureza_juridica.show(truncate=False)

+------------------------+------------------------------------------------------------------+
|codigo_natureza_juridica|natureza_juridica                                                 |
+------------------------+------------------------------------------------------------------+
|0000                    |Natureza Jurídica não informada                                   |
|3271                    |Órgão de Direção Local de Partido Político                        |
|3280                    |Comitê Financeiro de Partido Político                             |
|3298                    |Frente Plebiscitária ou Referendária                              |
|3301                    |Organização Social (OS)                                           |
|3999                    |Associação Privada                                                |
|4014                    |Empresa Individual Imobiliária                                    |
|4090                    |Candidato a Cargo Político Eletivo

In [ ]:
# Escrita dos dados no formato Delta, particionando por código

# gravar_delta(df_natureza_juridica, "../RAW/natureza_juridica")

Gravação efetuada!


DataFrame[codigo_natureza_juridica: string, natureza_juridica: string]

In [119]:
# Leitura da tabela Delta para validação
try:
    deltaTable_nj = DeltaTable.forPath(spark, "../RAW/natureza_juridica")
    df_natureza_juridica: DataFrame = deltaTable_nj.toDF()
    df_natureza_juridica.show(truncate=False)
except Exception as e:
    print(f"Erro: {e}")

+------------------------+------------------------------------------------------------------+
|codigo_natureza_juridica|natureza_juridica                                                 |
+------------------------+------------------------------------------------------------------+
|0000                    |Natureza Jurídica não informada                                   |
|3271                    |Órgão de Direção Local de Partido Político                        |
|3280                    |Comitê Financeiro de Partido Político                             |
|3298                    |Frente Plebiscitária ou Referendária                              |
|3301                    |Organização Social (OS)                                           |
|3999                    |Associação Privada                                                |
|4014                    |Empresa Individual Imobiliária                                    |
|4090                    |Candidato a Cargo Político Eletivo

## RAW - Empresas

In [120]:
# Caminho do arquivo CSV
path_csv_empresas: str = "../LND/CSV/empresas.csv"

In [123]:
# Definição do schema para o CSV de empresas
schema_empresas: StructType = StructType([
    StructField("cnpj_basico", StringType(), True),
    StructField("razao_social", StringType(), True),
    StructField("codigo_natureza_juridica", StringType(), True),
    StructField("codigo_qualificacao_responsavel", StringType(), True),
    StructField("capital_social", StringType(), True),
    StructField("codigo_porte_empresa", StringType(), True),
    StructField("ente_federativo_responsavel", StringType(), True)
])

In [124]:
# Leitura do arquivo CSV
df_empresas: DataFrame = ler_csv(path_csv_empresas, "iso-8859-1", schema_empresas)

In [125]:
df_empresas.show(5, truncate=False)

+-----------+--------------------------------------------------------+------------------------+-------------------------------+--------------+--------------------+---------------------------+
|cnpj_basico|razao_social                                            |codigo_natureza_juridica|codigo_qualificacao_responsavel|capital_social|codigo_porte_empresa|ente_federativo_responsavel|
+-----------+--------------------------------------------------------+------------------------+-------------------------------+--------------+--------------------+---------------------------+
|04631961   |ITACAMP INDUSTRIAL LTDA                                 |2062                    |49                             |0,00          |01                  |NULL                       |
|04631962   |CLINICA OFTALMOLOGICA DR VANIA XIMENES LTDA             |2062                    |49                             |0,00          |05                  |NULL                       |
|04631963   |EXPRESSO COSTA LTDA        

In [ ]:
#  Escrita no Delta Lake

# gravar_delta(df_empresas, "../RAW/empresas")

25/07/17 23:37:30 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/07/17 23:37:30 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/07/17 23:37:30 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
25/07/17 23:37:30 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/07/17 23:37:30 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
25/07/17 23:37:36 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/07/17 23:37:36 WARN MemoryManager: Total allocation exceeds 95.

Gravação efetuada!


DataFrame[cnpj_basico: string, razao_social: string, codigo_natureza_juridica: string, codigo_qualificacao_responsavel: string, capital_social: string, codigo_porte_empresa: string, ente_federativo_responsavel: string]

In [127]:
try:
    # Leitura e verificação da DeltaTable
    deltaTable_empresas = DeltaTable.forPath(spark, "../RAW/empresas")
    df_empresas: DataFrame = deltaTable_empresas.toDF()
    df_empresas.show(5, truncate=False)
except Exception as e:
    print(f"Erro ao ler DeltaTable empresas: {e}")

+-----------+---------------------------------------------------------------------+------------------------+-------------------------------+--------------+--------------------+---------------------------+
|cnpj_basico|razao_social                                                         |codigo_natureza_juridica|codigo_qualificacao_responsavel|capital_social|codigo_porte_empresa|ente_federativo_responsavel|
+-----------+---------------------------------------------------------------------+------------------------+-------------------------------+--------------+--------------------+---------------------------+
|08127452   |ELEICAO 2006 WALDEMAR ALBERTO BORGES RODRIGUES NETO DEPUTADO ESTADUAL|4090                    |51                             |0,00          |05                  |NULL                       |
|08127453   |ELEICAO 2006 JOSE ROBERTO DOS SANTOS PALADINI DEPUTADO ESTADUAL      |4090                    |51                             |0,00          |05                  |NULL

## RAW - Simples Nacional

In [ ]:
# Caminho CSV
path_csv_simples_nacional: str = '../LND/CSV/simples_nacional.csv'

In [132]:
# Schema do arquivo simples.csv
schema_simples_nacional: StructType = StructType([
    StructField("cnpj_basico", StringType(), True),
    StructField("opcao_simples", StringType(), True),
    StructField("data_opcao_simples", StringType(), True),
    StructField("data_exclusao_simples", StringType(), True),
    StructField("opcao_mei", StringType(), True),
    StructField("data_opcao_mei", StringType(), True),
    StructField("data_exclusao_mei", StringType(), True),
])

In [133]:
# # Leitura do CSV

df_simples_nacional = ler_csv(path_csv_simples_nacional, "iso-8859-1", schema_simples_nacional)

In [134]:
df_simples_nacional.show(5, truncate=False)

+-----------+-------------+------------------+---------------------+---------+--------------+-----------------+
|cnpj_basico|opcao_simples|data_opcao_simples|data_exclusao_simples|opcao_mei|data_opcao_mei|data_exclusao_mei|
+-----------+-------------+------------------+---------------------+---------+--------------+-----------------+
|00000000   |N            |20070701          |20070701             |N        |20090701      |20090701         |
|00000006   |N            |20180101          |20191231             |N        |00000000      |00000000         |
|00000008   |N            |20140101          |20211231             |N        |00000000      |00000000         |
|00000011   |S            |20070701          |00000000             |N        |00000000      |00000000         |
|00000013   |N            |20090101          |20231231             |N        |00000000      |00000000         |
+-----------+-------------+------------------+---------------------+---------+--------------+-----------

In [ ]:
# Escrita na camada RAW em Delta Lake

# gravar_delta(df_simples_nacional, "../RAW/simples_nacional")

25/07/17 23:41:57 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/07/17 23:41:57 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/07/17 23:41:57 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
25/07/17 23:41:57 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/07/17 23:41:57 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
25/07/17 23:42:23 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/07/17 23:42:23 WARN MemoryManager: Total allocation exceeds 95.

Gravação efetuada!


DataFrame[cnpj_basico: string, opcao_simples: string, data_opcao_simples: string, data_exclusao_simples: string, opcao_mei: string, data_opcao_mei: string, data_exclusao_mei: string]

In [136]:
try:
    deltaTable_simples: DeltaTable = DeltaTable.forPath(spark, "../RAW/simples_nacional")
    df_simples_nacional: DataFrame = deltaTable_simples.toDF()
    df_simples_nacional.show(5, truncate=False)
except Exception as e:
    print(f"Erro: {e}")

+-----------+-------------+------------------+---------------------+---------+--------------+-----------------+
|cnpj_basico|opcao_simples|data_opcao_simples|data_exclusao_simples|opcao_mei|data_opcao_mei|data_exclusao_mei|
+-----------+-------------+------------------+---------------------+---------+--------------+-----------------+
|36536474   |N            |20200303          |20201231             |N        |00000000      |00000000         |
|36536475   |S            |20200303          |00000000             |S        |20200303      |00000000         |
|36536476   |N            |20200303          |20220331             |N        |20200303      |20220331         |
|36536477   |N            |20200303          |20200403             |N        |20200303      |20200403         |
|36536478   |N            |20200303          |20241231             |N        |20200303      |20241231         |
+-----------+-------------+------------------+---------------------+---------+--------------+-----------

## RAW - Estabelecimentos

In [ ]:
# Caminho do CSV
path_csv_estabelecimentos: str = '../LND/CSV/estabelecimentos.csv'

In [ ]:
# Definição do Schema
schema_estabelecimentos: StructType = StructType([
    StructField("cnpj_base", StringType(), True),
    StructField("cnpj_ordem", StringType(), True),
    StructField("cnpj_dv", StringType(), True),
    StructField("matriz_filial", StringType(), True),
    StructField("nome_fantasia", StringType(), True),
    StructField("situacao_cadastral", StringType(), True),
    StructField("data_situacao_cadastral", StringType(), True),
    StructField("motivo_situacao_cadastral", StringType(), True),
    StructField("nome_cidade_exterior", StringType(), True),
    StructField("pais", StringType(), True),
    StructField("data_inicio_atividade", StringType(), True),
    StructField("cnae_fiscal", StringType(), True),
    StructField("cnae_fiscal_secundaria", StringType(), True),
    StructField("tipo_logradouro", StringType(), True),
    StructField("logradouro", StringType(), True),
    StructField("numero", StringType(), True),
    StructField("complemento", StringType(), True),
    StructField("bairro", StringType(), True),
    StructField("cep", StringType(), True),
    StructField("uf", StringType(), True),
    StructField("codigo_municipio", StringType(), True),
    StructField("ddd1", StringType(), True),
    StructField("telefone1", StringType(), True),
    StructField("ddd2", StringType(), True),
    StructField("telefone2", StringType(), True),
    StructField("ddd_fax", StringType(), True),
    StructField("fax", StringType(), True),
    StructField("email", StringType(), True),
    StructField("situacao_especial", StringType(), True),
    StructField("data_situacao_especial", StringType(), True)
])

In [ ]:
df_estabelecimentos: DataFrame = ler_csv(
    path_csv_estabelecimentos, 
    "iso-8859-1",
    schema_estabelecimentos
)

In [140]:
df_estabelecimentos.show(truncate=False)

+---------+----------+-------+-------------+-------------+------------------+-----------------------+-------------------------+--------------------+----+---------------------+-----------+----------------------+---------------+---------------------------+------+------------------+-------------------------+--------+---+----------------+----+---------+----+---------+-------+--------+-------------------------------------------+-----------------+----------------------+
|cnpj_base|cnpj_ordem|cnpj_dv|matriz_filial|nome_fantasia|situacao_cadastral|data_situacao_cadastral|motivo_situacao_cadastral|nome_cidade_exterior|pais|data_inicio_atividade|cnae_fiscal|cnae_fiscal_secundaria|tipo_logradouro|logradouro                 |numero|complemento       |bairro                   |cep     |uf |codigo_municipio|ddd1|telefone1|ddd2|telefone2|ddd_fax|fax     |email                                      |situacao_especial|data_situacao_especial|
+---------+----------+-------+-------------+-------------+----

In [ ]:
# Salvar na camada RAW

# gravar_delta(df_estabelecimentos, "../RAW/estabelecimentos")

25/07/17 23:44:55 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/07/17 23:44:55 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/07/17 23:44:55 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
25/07/17 23:44:55 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/07/17 23:44:55 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
25/07/17 23:45:26 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/07/17 23:45:26 WARN MemoryManager: Total allocation exceeds 95.

Gravação efetuada!


DataFrame[cnpj_base: string, cnpj_ordem: string, cnpj_dv: string, matriz_filial: string, nome_fantasia: string, situacao_cadastral: string, data_situacao_cadastral: string, motivo_situacao_cadastral: string, nome_cidade_exterior: string, pais: string, data_inicio_atividade: string, cnae_fiscal: string, cnae_fiscal_secundaria: string, tipo_logradouro: string, logradouro: string, numero: string, complemento: string, bairro: string, cep: string, uf: string, codigo_municipio: string, ddd1: string, telefone1: string, ddd2: string, telefone2: string, ddd_fax: string, fax: string, email: string, situacao_especial: string, data_situacao_especial: string]

In [142]:
try:
    deltaTable_estabelecimentos = DeltaTable.forPath(spark, "../RAW/estabelecimentos")
    df_estabelecimentos = deltaTable_estabelecimentos.toDF()
    df_estabelecimentos.show(5, truncate=False)
except Exception as e:
    print(f"Erro: {e}")

+---------+----------+-------+-------------+---------------------------------+------------------+-----------------------+-------------------------+--------------------+----+---------------------+-----------+-------------------------------------------------------+---------------+----------------------------+------+-----------------------+---------------------------+--------+---+----------------+----+---------+----+---------+-------+----+--------------------------------------+-----------------+----------------------+
|cnpj_base|cnpj_ordem|cnpj_dv|matriz_filial|nome_fantasia                    |situacao_cadastral|data_situacao_cadastral|motivo_situacao_cadastral|nome_cidade_exterior|pais|data_inicio_atividade|cnae_fiscal|cnae_fiscal_secundaria                                 |tipo_logradouro|logradouro                  |numero|complemento            |bairro                     |cep     |uf |codigo_municipio|ddd1|telefone1|ddd2|telefone2|ddd_fax|fax |email                                 |

## RAW - Socios

In [143]:
# Caminho do csv
path_csv_socios: str = "../LND/CSV/socios.csv"

In [144]:
# Definição do schema
schema_socios: StructType = StructType([
    StructField("cnpj_basico", StringType(), True),
    StructField("identificador_socio", StringType(), True),
    StructField("nome_socio_razao_social", StringType(), True),
    StructField("cnpj_cpf_socio", StringType(), True),
    StructField("codigo_qualificacao_socio", StringType(), True),
    StructField("data_entrada_sociedade", StringType(), True),
    StructField("pais", StringType(), True),
    StructField("representante_legal", StringType(), True),
    StructField("nome_representante", StringType(), True),
    StructField("codigo_qualificacao_representante", StringType(), True),
    StructField("faixa_etaria", StringType(), True),
])

In [145]:
# Ler csv
df_socios = ler_csv(path=path_csv_socios, encoding="iso-8859-1", schema=schema_socios)

In [146]:
df_socios.show(5, truncate=False)

+-----------+-------------------+-------------------------------+--------------+-------------------------+----------------------+----+-------------------+------------------+---------------------------------+------------+
|cnpj_basico|identificador_socio|nome_socio_razao_social        |cnpj_cpf_socio|codigo_qualificacao_socio|data_entrada_sociedade|pais|representante_legal|nome_representante|codigo_qualificacao_representante|faixa_etaria|
+-----------+-------------------+-------------------------------+--------------+-------------------------+----------------------+----+-------------------+------------------+---------------------------------+------------+
|41300034   |2                  |RONALDO ARMANDO                |***689697**   |49                       |20230727              |NULL|***000000**        |NULL              |00                               |7           |
|41300034   |2                  |RAVEL DA SILVA ARMANDO         |***189577**   |49                       |20230727  

In [ ]:
# salvar na camada RAW

# gravar_delta(df_socios, "../RAW/socios")

25/07/17 23:48:07 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/07/17 23:48:07 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/07/17 23:48:07 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
25/07/17 23:48:07 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/07/17 23:48:07 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
25/07/17 23:48:11 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/07/17 23:48:12 WARN MemoryManager: Total allocation exceeds 95.

Gravação efetuada!


DataFrame[cnpj_basico: string, identificador_socio: string, nome_socio_razao_social: string, cnpj_cpf_socio: string, codigo_qualificacao_socio: string, data_entrada_sociedade: string, pais: string, representante_legal: string, nome_representante: string, codigo_qualificacao_representante: string, faixa_etaria: string]

In [148]:
try:
    deltaTable_socios = DeltaTable.forPath(spark, "../RAW/socios")
    df_socios = deltaTable_socios.toDF()
    df_socios.show()
except Exception as e:
    print(f"Erro na leitura: {e}")

+-----------+-------------------+-----------------------+--------------+-------------------------+----------------------+----+-------------------+------------------+---------------------------------+------------+
|cnpj_basico|identificador_socio|nome_socio_razao_social|cnpj_cpf_socio|codigo_qualificacao_socio|data_entrada_sociedade|pais|representante_legal|nome_representante|codigo_qualificacao_representante|faixa_etaria|
+-----------+-------------------+-----------------------+--------------+-------------------------+----------------------+----+-------------------+------------------+---------------------------------+------------+
|   15508939|                  2|   LUIZ FERNANDO ROD...|   ***784978**|                       49|              20120423|NULL|        ***000000**|              NULL|                               00|           5|
|   15508939|                  2|   LUIZ GONZAGA RODR...|   ***555608**|                       49|              20120423|NULL|        ***000000**|  

## RAW - Paises

In [149]:
# Caminho do arquivo
path_csv_paises: str = "../LND/CSV/paises.csv"

In [150]:
# Definição do schema
schema_pais: StructType = StructType([
    StructField("codigo_pais", StringType(), True),
    StructField("nome_pais", StringType(), True),
])

In [151]:
# Ler arquivo CSV
df_paises = ler_csv(path_csv_paises, "iso-8859-1", schema_natureza_juridica)

In [152]:
# Mostrar resultado
df_paises.show(truncate=False)

+------------------------+-----------------------------+
|codigo_natureza_juridica|natureza_juridica            |
+------------------------+-----------------------------+
|000                     |COLIS POSTAUX                |
|013                     |AFEGANISTAO                  |
|017                     |ALBANIA                      |
|020                     |ALBORAN-PEREJIL,ILHAS        |
|023                     |ALEMANHA                     |
|025                     |ALEMANHA, REP. DEMOCRATICA DA|
|031                     |BURKINA FASO                 |
|037                     |ANDORRA                      |
|040                     |ANGOLA                       |
|041                     |ANGUILLA                     |
|043                     |ANTIGUA E BARBUDA            |
|047                     |ANTILHAS HOLANDESAS          |
|053                     |ARABIA SAUDITA               |
|059                     |ARGELIA                      |
|063                     |ARGEN

In [ ]:
# Gravar no RAW

# gravar_delta(df_paises, "../RAW/paises")

Gravação efetuada!


DataFrame[codigo_natureza_juridica: string, natureza_juridica: string]

In [155]:
try:
    # Transformar em DeltaTable e mostrar arquivo
    deltaTable_paises = DeltaTable.forPath(spark, "../RAW/paises")
    df_paises = deltaTable_paises.toDF()
    df_paises.show(truncate=False)
except Exception as e:
    print(f"Erro: {e}")

+------------------------+-----------------------------+
|codigo_natureza_juridica|natureza_juridica            |
+------------------------+-----------------------------+
|000                     |COLIS POSTAUX                |
|013                     |AFEGANISTAO                  |
|017                     |ALBANIA                      |
|020                     |ALBORAN-PEREJIL,ILHAS        |
|023                     |ALEMANHA                     |
|025                     |ALEMANHA, REP. DEMOCRATICA DA|
|031                     |BURKINA FASO                 |
|037                     |ANDORRA                      |
|040                     |ANGOLA                       |
|041                     |ANGUILLA                     |
|043                     |ANTIGUA E BARBUDA            |
|047                     |ANTILHAS HOLANDESAS          |
|053                     |ARABIA SAUDITA               |
|059                     |ARGELIA                      |
|063                     |ARGEN

## RAW - Qualificações

In [172]:
# Caminho do arquivo
path_csv_qualificacoes: str = "../LND/CSV/qualificacoes.csv"

In [173]:
# Definição do schema
schema_qualificacoes: StructType = StructType([
    StructField("codigo_qualificacao", StringType(), True),
    StructField("descricao_qualificacao", StringType(), True),
])

In [174]:
# ler arquivo CSV
df_qualificacoes = ler_csv(path_csv_qualificacoes, "iso-8859-1", schema_qualificacoes)

In [175]:
df_qualificacoes.show(truncate=False)

+-------------------+----------------------------+
|codigo_qualificacao|descricao_qualificacao      |
+-------------------+----------------------------+
|00                 |Não informada               |
|05                 |Administrador               |
|08                 |Conselheiro de Administração|
|09                 |Curador                     |
|10                 |Diretor                     |
|11                 |Interventor                 |
|12                 |Inventariante               |
|13                 |Liquidante                  |
|14                 |Mãe                         |
|15                 |Pai                         |
|16                 |Presidente                  |
|17                 |Procurador                  |
|18                 |Secretário                  |
|19                 |Síndico (Condomínio)        |
|20                 |Sociedade Consorciada       |
|21                 |Sociedade Filiada           |
|22                 |Sócio     

In [177]:
# Salvar no RAW

gravar_delta(df_qualificacoes, "../RAW/qualificacoes")

Gravação efetuada!


DataFrame[codigo_qualificacao: string, descricao_qualificacao: string]

In [178]:
try:
    deltaTable_qualificacoes = DeltaTable.forPath(spark, "../RAW/qualificacoes")
    df_qualificacoes = deltaTable_qualificacoes.toDF()
    df_qualificacoes.show(truncate=False)
except Exception as e:
    print(f"Erro: {e}")

+-------------------+----------------------------+
|codigo_qualificacao|descricao_qualificacao      |
+-------------------+----------------------------+
|00                 |Não informada               |
|05                 |Administrador               |
|08                 |Conselheiro de Administração|
|09                 |Curador                     |
|10                 |Diretor                     |
|11                 |Interventor                 |
|12                 |Inventariante               |
|13                 |Liquidante                  |
|14                 |Mãe                         |
|15                 |Pai                         |
|16                 |Presidente                  |
|17                 |Procurador                  |
|18                 |Secretário                  |
|19                 |Síndico (Condomínio)        |
|20                 |Sociedade Consorciada       |
|21                 |Sociedade Filiada           |
|22                 |Sócio     

## RAW - Motivos

In [162]:
# caminho do CSV
path_csv_motivos: str = "../LND/CSV/motivos.csv"

In [163]:
# Definição do schema
schema_motivos: StructType = StructType([
    StructField("codigo_motivo", StringType(), True),
    StructField("descricao_motivo", StringType(), True),
]) 

In [164]:
# Ler arquivo CSV
df_motivos: DataFrame = ler_csv(path_csv_motivos, "iso-8859-1", schema_motivos)

In [165]:
df_motivos.show(truncate=False)

+-------------+-------------------------------------------------+
|codigo_motivo|descricao_motivo                                 |
+-------------+-------------------------------------------------+
|00           |SEM MOTIVO                                       |
|01           |EXTINCAO POR ENCERRAMENTO LIQUIDACAO VOLUNTARIA  |
|02           |INCORPORACAO                                     |
|03           |FUSAO                                            |
|04           |CISAO TOTAL                                      |
|05           |ENCERRAMENTO DA FALENCIA                         |
|06           |ENCERRAMENTO DA LIQUIDACAO                       |
|07           |ELEVACAO A MATRIZ                                |
|08           |TRANSPASSE                                       |
|09           |NAO INICIO DE ATIVIDADE                          |
|10           |EXTINCAO PELO ENCERRAMENTO DA LIQUIDACAO JUDICIAL|
|11           |ANULACAO POR MULTICIPLIDADE                      |
|12       

In [ ]:
# Gravar no RAW

# gravar_delta(df_motivos, "../RAW/motivos")

Gravação efetuada!


DataFrame[codigo_motivo: string, descricao_motivo: string]

In [167]:
try:
    deltaTable_motivos: DeltaTable = DeltaTable.forPath(spark, "../RAW/motivos")
    df_motivos: DataFrame = deltaTable_motivos.toDF()
    df_motivos.show(truncate=False)
except Exception as e:
    print(f"Erro: {e}")

+-------------+-------------------------------------------------+
|codigo_motivo|descricao_motivo                                 |
+-------------+-------------------------------------------------+
|00           |SEM MOTIVO                                       |
|01           |EXTINCAO POR ENCERRAMENTO LIQUIDACAO VOLUNTARIA  |
|02           |INCORPORACAO                                     |
|03           |FUSAO                                            |
|04           |CISAO TOTAL                                      |
|05           |ENCERRAMENTO DA FALENCIA                         |
|06           |ENCERRAMENTO DA LIQUIDACAO                       |
|07           |ELEVACAO A MATRIZ                                |
|08           |TRANSPASSE                                       |
|09           |NAO INICIO DE ATIVIDADE                          |
|10           |EXTINCAO PELO ENCERRAMENTO DA LIQUIDACAO JUDICIAL|
|11           |ANULACAO POR MULTICIPLIDADE                      |
|12       

## RAW - Municipios Receita Federal

In [4]:
# caminho do CSV
path_csv_municipios_rf: str = "../LND/CSV/municipios.csv"

In [12]:
schema_rf_municipios: StructType = StructType([
    StructField("id_municipio", StringType(), True),
    StructField("municipio", StringType(), True)
])

In [13]:
# Ler arquivo CSV
df_municipios_rf: DataFrame = ler_csv(path_csv_municipios_rf, "iso-8859-1", schema_rf_municipios)

In [14]:
df_municipios_rf.show(5, truncate=False)
df_municipios_rf.printSchema()

+------------+-----------------------+
|id_municipio|municipio              |
+------------+-----------------------+
|0001        |GUAJARA-MIRIM          |
|0002        |ALTO ALEGRE DOS PARECIS|
|0003        |PORTO VELHO            |
|0004        |BURITIS                |
|0005        |JI-PARANA              |
+------------+-----------------------+
only showing top 5 rows
root
 |-- id_municipio: string (nullable = true)
 |-- municipio: string (nullable = true)



In [16]:
# Gravar no RAW

gravar_delta(df_municipios_rf, "../RAW/municipios_rf")

25/07/18 23:19:29 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Gravação efetuada!


DataFrame[id_municipio: string, municipio: string]

In [17]:
try:
    deltaTable_municipios_rf: DeltaTable = DeltaTable.forPath(spark, "../RAW/municipios_rf")
    df_municipios_rf: DataFrame = deltaTable_municipios_rf.toDF()
    df_municipios_rf.show(truncate=False)
    df_municipios_rf.printSchema()
except Exception as e:
    print(f"Erro: {e}")

+------------+------------------------+
|id_municipio|municipio               |
+------------+------------------------+
|0001        |GUAJARA-MIRIM           |
|0002        |ALTO ALEGRE DOS PARECIS |
|0003        |PORTO VELHO             |
|0004        |BURITIS                 |
|0005        |JI-PARANA               |
|0006        |CHUPINGUAIA             |
|0007        |ARIQUEMES               |
|0008        |CUJUBIM                 |
|0009        |CACOAL                  |
|0010        |NOVA UNIAO              |
|0011        |PIMENTA BUENO           |
|0012        |PARECIS                 |
|0013        |VILHENA                 |
|0014        |PIMENTEIRAS DO OESTE    |
|0015        |JARU                    |
|0016        |PRIMAVERA DE RONDONIA   |
|0017        |OURO PRETO DO OESTE     |
|0018        |SAO FELIPE D'OESTE      |
|0019        |PRESIDENTE MEDICI       |
|0020        |SAO FRANCISCO DO GUAPORE|
+------------+------------------------+
only showing top 20 rows
root
 |-- id_mu

## Finalizar spark

In [3]:
spark.stop()